In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
#载入数据集
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

#运行次数
max_steps=1001
#图片数量
image_num=3000
#文件路径
DIR='/home/yejg/'

#定义会话
sess=tf.Session()

#载入图片
embedding=tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

#定义一个函数
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#记录平均值
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        

#定义两个占位符

with tf.name_scope('input'):
    x=tf.placeholder(shape=[None,784],dtype=tf.float32,name='x_input')
    y=tf.placeholder(shape=[None,10],dtype=tf.float32,name='y_input')
    
#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input=tf.reshape(x,[-1,28,28,1])  #-1代表不确定的值，对于None,1代表黑白图片
    tf.summary.image('input',image_shaped_input,10)  #放10张图片

#创建一个简单的神经网络
with tf.name_scope('layer'):
    with tf.name_scope('weight'):
        w=tf.Variable(tf.zeros(shape=[784,10]))
        variable_summaries(w)
    with tf.name_scope('biases'):
        b=tf.Variable(tf.zeros(shape=[10]))
        variable_summaries(b)
    with tf.name_scope('preidicton'):
        prediction=tf.nn.softmax(tf.matmul(x,w)+b)
        
        
#二次代价函数
with tf.name_scope('train'):
    with tf.name_scope('loss'):
       loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                   labels=y,logits=prediction))
       tf.summary.scalar('loss',loss)
    with tf.name_scope('train_step'):
       #使用梯度下降法
       train_step=tf.train.GradientDescentOptimizer(0.2).minimize(loss)

        
sess.run(tf.global_variables_initializer())

#结果存在一个bool列表中
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
         correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    
    with tf.name_scope('accuracy'):
         #求准确率
         accuracy=tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))
         tf.summary.scalar('accuracy',accuracy)
            
#产生metadata文件
if tf.gfile.Exists(DIR+'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR+'projector/projector/metadata.tsv')
with open(DIR+'projector/projector/metadata.tsv','w') as f:
    labels=sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i])+'\n')
        
#合并所有的summary
merged=tf.summary.merge_all()

projector_writer=tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
saver=tf.train.Saver()

config=projector.ProjectorConfig()
embed=config.embeddings.add()
embed.tensor.name=embedding.name
embed.metadata_path=DIR+'projector/projector/metadata.tsv'
embed.sprite.image_path=DIR+'mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)


for i in range(max_steps):
    batch_xs,batch_ys=mnist.train.next_batch(100)
    run_options=tf.RunOptions(trace_level=tf.Options.FULL_TRACE)
    run_metadata=tf.RunMetadata()
    summary,_=sess.run([merged,train_step],feed_dict={
                  x:batch_xs,
                  y:batch_y
    })
    
    projector_writer.add_run_metadata(run_metadata,'step%03d'%i)
    projector_writer.add_summary(summary,i)
    
    if i%100==0:
        acc=sess.run(accuracy,feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels
        })
        print('Iter'+str(i)+',Testing Accuracy='+str(acc))
        
saver.save(sess,DIR+'projector/projector/a_model.ckpt',global_step=max_stpes)
projector_writer.close()
sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


FailedPreconditionError: /home/yejg/projector/projector/metadata.tsv